<a href="https://colab.research.google.com/github/ayushraj16/SnakeGame---Gradio-Web-App/blob/main/snakeGame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gradio as gr
import json
import random
import time

class SnakeGame:
    def __init__(self, width=25, height=25):
        self.width = width
        self.height = height
        self.reset_game()

    def reset_game(self):
        """Reset the game to initial state"""
        self.snake = [(12, 12), (12, 13), (12, 14), (12, 15), (12, 16)]
        self.direction = (0, -1)  # Moving up
        self.food = self.generate_food()
        self.score = 0
        self.game_over = False
        self.game_started = False

    def generate_food(self):
        """Generate random food position not on snake"""
        while True:
            food = (random.randint(0, self.width-1), random.randint(0, self.height-1))
            if food not in self.snake:
                return food

    def move(self, direction=None):
        """Move snake in given direction"""
        if self.game_over or not self.game_started:
            return

        if direction:
            # Prevent moving in opposite direction
            if (direction[0] * -1, direction[1] * -1) != self.direction:
                self.direction = direction

        # Calculate new head position
        head = self.snake[-1]
        new_head = (
            (head[0] + self.direction[0]) % self.width,
            (head[1] + self.direction[1]) % self.height
        )

        # Check collision with self
        if new_head in self.snake:
            self.game_over = True
            return

        # Add new head
        self.snake.append(new_head)

        # Check food collision
        if new_head == self.food:
            self.score += 10
            self.food = self.generate_food()
        else:
            # Remove tail if no food eaten
            self.snake.pop(0)

    def get_board_html(self):
        """Generate HTML representation of the game board"""
        board = [['.' for _ in range(self.width)] for _ in range(self.height)]

        # Place snake
        for i, (x, y) in enumerate(self.snake):
            if i == len(self.snake) - 1:  # Head
                board[y][x] = '🟢'
            else:  # Body
                board[y][x] = '🟩'

        # Place food
        fx, fy = self.food
        board[fy][fx] = '🍎'

        # Convert to HTML
        html = """
        <div style="font-family: monospace; font-size: 16px; line-height: 1.2;
                    background: #000; padding: 20px; border-radius: 10px;
                    display: inline-block; border: 3px solid #444;">
        """

        for row in board:
            html += '<div style="margin: 1px 0;">'
            for cell in row:
                if cell == '.':
                    html += '<span style="color: #333;">⬛</span>'
                else:
                    html += cell
            html += '</div>'

        html += '</div>'

        if self.game_over:
            html += '<div style="color: red; font-size: 24px; font-weight: bold; margin-top: 10px;">GAME OVER!</div>'
        elif not self.game_started:
            html += '<div style="color: yellow; font-size: 20px; margin-top: 10px;">Press START to begin!</div>'

        return html

# Global game instance
game = SnakeGame()

def start_game():
    """Start a new game"""
    game.reset_game()
    game.game_started = True
    return game.get_board_html(), f"Score: {game.score}", "Game Started! Use arrow buttons to move."

def move_up():
    """Move snake up"""
    game.move((0, -1))
    status = "Game Over!" if game.game_over else "Playing..."
    return game.get_board_html(), f"Score: {game.score}", status

def move_down():
    """Move snake down"""
    game.move((0, 1))
    status = "Game Over!" if game.game_over else "Playing..."
    return game.get_board_html(), f"Score: {game.score}", status

def move_left():
    """Move snake left"""
    game.move((-1, 0))
    status = "Game Over!" if game.game_over else "Playing..."
    return game.get_board_html(), f"Score: {game.score}", status

def move_right():
    """Move snake right"""
    game.move((1, 0))
    status = "Game Over!" if game.game_over else "Playing..."
    return game.get_board_html(), f"Score: {game.score}", status

def auto_move():
    """Continue moving in current direction"""
    if game.game_started and not game.game_over:
        game.move()
    status = "Game Over!" if game.game_over else ("Playing..." if game.game_started else "Not Started")
    return game.get_board_html(), f"Score: {game.score}", status

def get_game_info():
    """Get current game information"""
    return {
        "Score": game.score,
        "Snake Length": len(game.snake),
        "Food Position": f"({game.food[0]}, {game.food[1]})",
        "Game Status": "Game Over" if game.game_over else ("Playing" if game.game_started else "Not Started")
    }

# Create Gradio interface
def create_interface():
    with gr.Blocks(
        title="🐍 Snake Game",
        theme=gr.themes.Monochrome(),
        css="""
        .game-container { text-align: center; }
        .control-buttons { margin: 10px; }
        .game-board { margin: 20px auto; }
        """
    ) as demo:

        gr.Markdown("# 🐍 Snake Game")
        gr.Markdown("Control the snake to eat food and grow longer. Avoid hitting yourself!")

        with gr.Row():
            with gr.Column(scale=2):
                # Game board display
                board_display = gr.HTML(
                    value=game.get_board_html(),
                    elem_classes=["game-board"]
                )

                # Game controls
                with gr.Row():
                    start_btn = gr.Button("🚀 START GAME", variant="primary", size="lg")

                with gr.Row():
                    gr.Button("", visible=False)  # Spacer
                    up_btn = gr.Button("⬆️ UP", size="sm")
                    gr.Button("", visible=False)  # Spacer

                with gr.Row():
                    left_btn = gr.Button("⬅️ LEFT", size="sm")
                    auto_btn = gr.Button("🎯 AUTO MOVE", size="sm")
                    right_btn = gr.Button("➡️ RIGHT", size="sm")

                with gr.Row():
                    gr.Button("", visible=False)  # Spacer
                    down_btn = gr.Button("⬇️ DOWN", size="sm")
                    gr.Button("", visible=False)  # Spacer

            with gr.Column(scale=1):
                # Game statistics
                gr.Markdown("## 📊 Game Stats")
                score_display = gr.Textbox(
                    label="Score",
                    value="Score: 0",
                    interactive=False
                )

                status_display = gr.Textbox(
                    label="Status",
                    value="Press START to begin!",
                    interactive=False
                )

                # Game info
                gr.Markdown("## ℹ️ Game Info")
                info_display = gr.JSON(
                    value=get_game_info(),
                    label="Current Game State"
                )

                # Instructions
                gr.Markdown("""
                ## 🎮 How to Play
                - 🍎 Eat red apples to grow
                - 🟢 Green circle is snake head
                - 🟩 Green squares are snake body
                - ⬛ Black squares are empty space
                - Snake wraps around edges
                - Don't hit yourself!

                ## 🎯 Tips
                - Use AUTO MOVE for continuous movement
                - Plan your path carefully
                - Try to beat your high score!
                """)

        # Button event handlers
        start_btn.click(
            fn=start_game,
            outputs=[board_display, score_display, status_display]
        ).then(
            fn=get_game_info,
            outputs=[info_display]
        )

        up_btn.click(
            fn=move_up,
            outputs=[board_display, score_display, status_display]
        ).then(
            fn=get_game_info,
            outputs=[info_display]
        )

        down_btn.click(
            fn=move_down,
            outputs=[board_display, score_display, status_display]
        ).then(
            fn=get_game_info,
            outputs=[info_display]
        )

        left_btn.click(
            fn=move_left,
            outputs=[board_display, score_display, status_display]
        ).then(
            fn=get_game_info,
            outputs=[info_display]
        )

        right_btn.click(
            fn=move_right,
            outputs=[board_display, score_display, status_display]
        ).then(
            fn=get_game_info,
            outputs=[info_display]
        )

        auto_btn.click(
            fn=auto_move,
            outputs=[board_display, score_display, status_display]
        ).then(
            fn=get_game_info,
            outputs=[info_display]
        )

    return demo

# Launch the app
if __name__ == "__main__":
    demo = create_interface()
    demo.launch(
        share=True,
        server_name="0.0.0.0",
        server_port=7860,
        show_error=True
    )